<a href="https://colab.research.google.com/github/pnkjjha/AAS/blob/main/Structured_Streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=143d725f1ac0a3245103b8fe82634bcbdf6669df3220833f2a1324195850bb40
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pandas import json_normalize
import pyspark.sql.functions as F
import pyspark.sql.types as T
import random
import requests
import json
import os
import pandas as pd
import datetime

In [180]:
spark=SparkSession.builder.appName('test').config("spark.sql.shuffle.partitions", "4").getOrCreate()

In [ ]:
# url = "https://api.carbonintensity.org.uk/intensity"

# while switch:
#   response = requests.get(url)
#   json_normalize(response.json())
#   message = response.json()
#   json_normalize(message['data']).rename(columns={"intensity.forecast": "intensity_forecast",\
#                                                   "intensity.actual": "intensity_actual",\
#                                                   "intensity.percentile": "intensity_percentile",\
#                                                   "intensity.index": "intensity_index"\
#                                                   }).to_csv("intensity_"+str(random.randint(1,10000))+".csv", index=False)


In [4]:
# read lines from file
def flat_json(line):
  # convert line into json
  line_json = json.loads(line)
# json normalize
  flatline = pd.json_normalize(line_json,sep="_")
  return flatline


#Inputfile generation

In [181]:
%rm -rf input
%mkdir input

In [5]:
datetime.datetime.strftime(datetime.datetime.utcnow(),"%Y-%m-%d %H:%M:%S")

'2024-09-11 10:47:15'

In [94]:
i = 0
s = requests.Session()
url = "https://stream.wikimedia.org/v2/stream/recentchange"
file_number = str(random.randint(1,10000))
with s.get(url, stream=True) as resp:
    for line in resp.iter_lines(1024,decode_unicode=True):
        # Check if line has some bytes in it
        if line:
            # convert line into json
            # get data lines only
            if line[:4] == "data":
                # print(line[5:])
                json_line = json.loads(line[5:])
                flatlines = pd.json_normalize(json_line,sep="_")
                with open("input/rc"+str(random.randint(1,10000))+".json","a",encoding="utf-8") as f:
                    flatlines.to_json(f,orient="records",lines=True,index=False)
                    i = i + 1
                    if i>=9000 :
                        break

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [182]:
%ls -l input | wc -l

1


In [ ]:
flat_json_pdf["meta_dt"].head(2)

,meta_dt
0,2024-09-10T06:37:03Z


In [183]:
# design schema here
dSchema = T.StructType([\
   T.StructField("$Schema", T.StringType())\
  ,T.StructField("id", T.IntegerType())\
  ,T.StructField("type", T.StringType())\
  ,T.StructField("namespace", T.IntegerType())\
  ,T.StructField("title", T.StringType())\
  ,T.StructField("title_url", T.StringType())\
  ,T.StructField("comment", T.StringType())\
  ,T.StructField("timestamp", T.IntegerType())\
  ,T.StructField("user", T.StringType())\
  ,T.StructField("bot", T.BooleanType())\
  ,T.StructField("log_id", T.IntegerType())\
  ,T.StructField("log_type", T.StringType())\
  ,T.StructField("log_action", T.StringType())\
  ,T.StructField("log_action_comment", T.StringType())\
  ,T.StructField("server_url", T.StringType())\
  ,T.StructField("server_name", T.StringType())\
  ,T.StructField("server_script_path", T.StringType())\
  ,T.StructField("wiki", T.StringType())\
  ,T.StructField("parsedcomment", T.StringType())\
  ,T.StructField("meta_uri", T.StringType())\
  ,T.StructField("meta_request_id", T.StringType())\
  ,T.StructField("meta_id", T.StringType())\
  ,T.StructField("meta_dt", T.TimestampType())\
  ,T.StructField("meta_domain", T.StringType())\
  ,T.StructField("meta_stream", T.StringType())\
  ,T.StructField("meta_topic", T.StringType())\
  ,T.StructField("meta_partition", T.IntegerType())\
  ,T.StructField("meta_offset", T.IntegerType())\
  ,T.StructField("log_params_userid", T.IntegerType())\
                      ])

# Folder clean up

In [184]:
%rm -f json_output/*.json

In [185]:
%rm -rf *check*

In [187]:
%rm -rf *output*

In [ ]:
static_df= spark.read.load("input/recentChanges1612.json",format="json",schema=dSchema)

In [ ]:
static_df.select("meta_dt").show(15)

In [ ]:
static_df.dtypes

[('$Schema', 'string'),
 ('id', 'int'),
 ('type', 'string'),
 ('namespace', 'int'),
 ('title', 'string'),
 ('title_url', 'string'),
 ('comment', 'string'),
 ('timestamp', 'int'),
 ('user', 'string'),
 ('bot', 'boolean'),
 ('log_id', 'int'),
 ('log_type', 'string'),
 ('log_action', 'string'),
 ('log_action_comment', 'string'),
 ('server_url', 'string'),
 ('server_name', 'string'),
 ('server_script_path', 'string'),
 ('wiki', 'string'),
 ('parsedcomment', 'string'),
 ('meta_uri', 'string'),
 ('meta_request_id', 'string'),
 ('meta_id', 'string'),
 ('meta_dt', 'timestamp'),
 ('meta_domain', 'string'),
 ('meta_stream', 'string'),
 ('meta_topic', 'string'),
 ('meta_partition', 'int'),
 ('meta_offset', 'int'),
 ('log_params_userid', 'int')]

# Structured Streaming

In [8]:
#create Dataframe representing the stream of input lines from connection to localhost:9999
recentChange = spark.readStream.format("json").schema(dSchema).load("input")


In [9]:
recentChange.isStreaming

True

In [ ]:
rc_df = recentChange.withColumn("meta_dt", recentChange.meta_dt.cast(T.TimestampType()))
rc_df

DataFrame[$Schema: string, id: int, type: string, namespace: int, title: string, title_url: string, comment: string, timestamp: int, user: string, bot: boolean, log_id: int, log_type: string, log_action: string, log_action_comment: string, server_url: string, server_name: string, server_script_path: string, wiki: string, parsedcomment: string, meta_uri: string, meta_request_id: string, meta_id: string, meta_dt: timestamp, meta_domain: string, meta_stream: string, meta_topic: string, meta_partition: int, meta_offset: int, log_params_userid: int]

In [93]:
# Assuming 'recentChange' is your streaming DataFrame
myquery = recentChange\
  .withColumn("meta_dt", recentChange.meta_dt.cast(T.TimestampType()))\
  .withWatermark("meta_dt", "20 minutes")\
  .groupBy(F.window("meta_dt","2 minutes")\
         , "type")\
  .count()\
  .writeStream\
  .format("csv")\
  .outputMode("append")\
  .option("path", "output")\
  .option("header","true")\
  .option("checkpointLocation", "checkpoint_dir")\
  .start()



In [102]:
%ls -lrt output

total 4
drwxr-xr-x 2 root root 4096 Sep 11 11:31 _spark_metadata/


In [103]:
myquery.stop()

In [ ]:
inmemoryQuery = recentChange\
  .withColumn("meta_dt", recentChange.meta_dt.cast(T.TimestampType()))\
  .groupBy(F.window("meta_dt","1 minutes"), "type")\
  .count()\
  .writeStream\
  .queryName("myquery")\
  .format("memory")\
  .outputMode("complete")\
  .start()

In [ ]:
spark.sql("select * from myquery order by type").show(truncate=False)

In [56]:
inmemoryQuery.stop()

In [ ]:
%ls -lr input

#Testing

In [202]:
rate_df = spark.readStream.format("rate").option("rowsPerSecond", 100).load()

In [189]:
rate_df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- value: long (nullable = true)



In [ ]:
rate_df.writeStream.format("memory").queryName("query_name").outputMode("append").start()

In [ ]:
spark.sql("select * from query_name").show(truncate=False)

In [203]:
json_rate_df=rate_df.withWatermark("timestamp", "5 minutes")\
.groupBy(F.window("timestamp","1 minutes"))\
.count()\
.writeStream.format("json").option("path", "json_output").option("checkpointLocation", "checkpoint_dir_json")\
.trigger(processingTime="3 minutes")\
.start()

In [191]:
datetime.datetime.strftime(datetime.datetime.utcnow(),"%Y-%m-%d %H:%M:%S")

'2024-09-11 15:05:24'

In [213]:
%ls --full-time -lrt  json_output

total 40
-rw-r--r-- 1 root root    0 2024-09-11 15:05:18.714928525 +0000 part-00000-a7dd3eee-c968-4ddd-b598-24d4d69344ae-c000.json
-rw-r--r-- 1 root root    0 2024-09-11 15:06:00.222929238 +0000 part-00000-80440d50-86ca-404f-8480-b8f07b5909a9-c000.json
-rw-r--r-- 1 root root    0 2024-09-11 15:09:00.213277456 +0000 part-00000-a05b2607-333f-4d34-a794-94d039c465f0-c000.json
-rw-r--r-- 1 root root    0 2024-09-11 15:11:50.316672729 +0000 part-00000-4ad91a5d-7aa1-4475-a033-03dfca0769bb-c000.json
-rw-r--r-- 1 root root    0 2024-09-11 15:12:00.343639167 +0000 part-00000-af86b9c7-5a08-4c6d-ac8c-c4b9d5b80840-c000.json
-rw-r--r-- 1 root root   92 2024-09-11 15:12:00.485652855 +0000 part-00002-fcd693e7-aec8-438c-b5d0-c0fad4df0dd4-c000.json
-rw-r--r-- 1 root root    0 2024-09-11 15:15:00.203974861 +0000 part-00000-0fbea853-a675-4bc2-90e1-e965fe3a0983-c000.json
-rw-r--r-- 1 root root   92 2024-09-11 15:18:00.554357779 +0000 part-00001-29524609-ada2-431e-a86c-2c7e860bd465-c000.json
-rw-r--r-- 1 ro

In [215]:
json_rate_df.stop()

In [214]:
%cat json_output/part-00002-70c4bd8c-ed49-402f-a95a-6c4545de077e-c000.json

{"window":{"start":"2024-09-11T15:13:00.000Z","end":"2024-09-11T15:14:00.000Z"},"count":6000}
{"window":{"start":"2024-09-11T15:14:00.000Z","end":"2024-09-11T15:15:00.000Z"},"count":6000}


In [176]:
%cat json_output/part-00091-42089495-424d-444d-9ede-f083acfd61dd-c000.json

{"window":{"start":"2024-09-11T14:27:00.000Z","end":"2024-09-11T14:28:00.000Z"},"count":600}


In [177]:
%cat json_output/part-00075-f8c9962f-45e4-47ac-9a38-767593f31c98-c000.json

{"window":{"start":"2024-09-11T14:17:00.000Z","end":"2024-09-11T14:18:00.000Z"},"count":600}
